

```
# This is formatted as code
```

# DenseNet121

BCE Loss

Hyperparameters taken from libauc multilabel example

In [ ]:
# Mount into drive
from google.colab import drive
drive.mount("/content/drive")
!cp '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/CheXpert-v1.0-small.zip' /content/
!mkdir CheXpert
!unzip '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/CheXpert-v1.0-small.zip' -d /content/CheXpert
import sys
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/tyler')
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation')
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/models')

In [ ]:
# imports
!pip install transformers
!pip install libauc
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import torch
print(torch.cuda.is_available())

from transformers import AutoFeatureExtractor, SwinForImageClassification, SwinConfig
from transformers import ConvNextFeatureExtractor, ConvNextForImageClassification
from transformers import TrainingArguments, Trainer
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop
from torchvision.transforms.functional import InterpolationMode
import torchvision.models as models
from libauc.models import DenseNet121, DenseNet169

from sklearn.model_selection import train_test_split

In [ ]:

from dataloader import *
from utils import *
from trainer import *

In [ ]:
from libauc.losses import AUCM_MultiLabel, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import DenseNet121, DenseNet169
from libauc.datasets import CheXpert

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F   

In [ ]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# dataloader
root = '/content/CheXpert/CheXpert-v1.0-small/'
# Index: -1 denotes multi-label mode including 5 diseases
traindSet = CheXpert(csv_path=root+'train.csv', image_root_path=root, use_upsampling=False, use_frontal=False, image_size=224, mode='train', class_index=-1, verbose=False)
testSet =  CheXpert(csv_path=root+'valid.csv',  image_root_path=root, use_upsampling=False, use_frontal=False, image_size=224, mode='valid', class_index=-1, verbose=False)
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=32, num_workers=2, shuffle=True)
testloader =  torch.utils.data.DataLoader(testSet, batch_size=32, num_workers=2, shuffle=False)

# paramaters
SEED = 123
BATCH_SIZE = 32
 
lr = 0.001 # using smaller learning rate is better
gamma = 500
imratio = traindSet.imratio_list 
weight_decay = 1e-5
margin = 1.0

# model
set_all_seeds(SEED)
model = DenseNet121(pretrained=True, last_activation=None, activations='relu', num_classes=5)
model = model.cuda()

# define loss & optimizer
Loss = CrossEntropyLoss()
# Loss = AUCM_MultiLabel(imratio=imratio, num_classes=5)
optimizer = Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
# optimizer = PESG(model, 
#                  a=Loss.a, 
#                  b=Loss.b, 
#                  alpha=Loss.alpha, 
#                  lr=lr, 
#                  gamma=gamma, 
#                  margin=margin, 
#                  weight_decay=weight_decay, device='cuda')


# training
best_val_auc = 0 
for epoch in range(10):
    # if epoch > 0:
    #     optimizer.update_regularizer(decay_factor=10)       
    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      # y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 400 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for jdx, data in enumerate(testloader):
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  y_pred = torch.sigmoid(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              model.train()

              if best_val_auc < val_auc_mean:
                 best_val_auc = val_auc_mean
                 torch.save(model.state_dict(), 'aucm_multi_label_pretrained_model.pth')

              print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, val_auc_mean, best_val_auc))

/usr/local/lib/python3.7/dist-packages/libauc/datasets/chexpert.py:35: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0-small/', '')
/usr/local/lib/python3.7/dist-packages/libauc/datasets/chexpert.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0/', '')


Epoch=0, BatchID=0, Val_AUC=0.5576, Best_Val_AUC=0.5576
Epoch=0, BatchID=400, Val_AUC=0.7694, Best_Val_AUC=0.7694
Epoch=0, BatchID=800, Val_AUC=0.8514, Best_Val_AUC=0.8514
Epoch=0, BatchID=1200, Val_AUC=0.8414, Best_Val_AUC=0.8514
Epoch=0, BatchID=1600, Val_AUC=0.8605, Best_Val_AUC=0.8605
Epoch=0, BatchID=2000, Val_AUC=0.8608, Best_Val_AUC=0.8608
Epoch=0, BatchID=2400, Val_AUC=0.8418, Best_Val_AUC=0.8608
Epoch=0, BatchID=2800, Val_AUC=0.8356, Best_Val_AUC=0.8608
Epoch=0, BatchID=3200, Val_AUC=0.8489, Best_Val_AUC=0.8608
Epoch=0, BatchID=3600, Val_AUC=0.8684, Best_Val_AUC=0.8684
Epoch=0, BatchID=4000, Val_AUC=0.8500, Best_Val_AUC=0.8684
Epoch=0, BatchID=4400, Val_AUC=0.8612, Best_Val_AUC=0.8684
Epoch=0, BatchID=4800, Val_AUC=0.8410, Best_Val_AUC=0.8684
Epoch=0, BatchID=5200, Val_AUC=0.8440, Best_Val_AUC=0.8684
Epoch=0, BatchID=5600, Val_AUC=0.7928, Best_Val_AUC=0.8684
Epoch=0, BatchID=6000, Val_AUC=0.8622, Best_Val_AUC=0.8684
Epoch=0, BatchID=6400, Val_AUC=0.8775, Best_Val_AUC=0.8775
Ep